## Libraries

In [77]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import concurrent.futures
import random
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

## Functions

In [2]:
def sp_symbols(api_key):
  url = f'https://financialmodelingprep.com/api/v3/sp500_constituent?apikey={api_key}'
  stocks = requests.get(url).json()
  symbol_list = []
  for i in range(len(stocks)):
    symbol_list.append(stocks[i]['symbol'])
  return symbol_list

In [4]:
def import_price_data(api_key,symbol):
    prices = requests.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={api_key}').json()
    prices = pd.DataFrame(prices['historical'])
    prices = prices.sort_values(by='date').reset_index(drop=True)
    return prices

In [5]:
def import_company_data(api_key,symbol):
    company = requests.get(f'https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={api_key}').json()
    company = pd.DataFrame(company)
    return company

In [96]:
def import_key_metric_data(symbol,api_key):
  key_metrics = requests.get(f'https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?period=quarterly&apikey={api_key}').json()
  key_metrics_df = pd.DataFrame(key_metrics)
  return key_metrics_df

In [97]:
def import_div_history(symbol,api_key):
  url = f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/{symbol}?apikey={api_key}"
  div_hist_df = pd.DataFrame(requests.get(url).json()['historical'])
  return div_hist_df

In [8]:
def merge_metric_and_div_data(metric_data,div_data):

  # Ensure date columns are datetime
  metric_data['date'] = pd.to_datetime(metric_data['date'])
  div_data['paymentDate'] = pd.to_datetime(div_data['paymentDate'])

  # Function to find the closest date
  def find_closest_date(date, date_series):
      return date_series.iloc[(date_series - date).abs().argsort().iloc[0]]

  # Apply the function to get the closest paymentDate for each date in metric_data_subset
  metric_data['closest_paymentDate'] = metric_data['date'].apply(lambda x: find_closest_date(x, div_data['paymentDate']))

  # Merge the dividend column based on the closest paymentDate
  metric_data = metric_data.merge(div_data[['paymentDate', 'dividend']],
                                  left_on='closest_paymentDate', right_on='paymentDate', how='left')

  # Drop the helper column
  metric_data = metric_data.drop(columns=['closest_paymentDate', 'paymentDate'])

  return metric_data

In [ ]:
def annualized_returns_and_sharpe(symbol,api_key):

  price_data = import_price_data(symbol = symbol,api_key = api_key)
  price_data['daily_return'] = price_data.adjClose.pct_change()
  price_data['annualized_daily_return'] = price_data.daily_return * 252

  annualized_return = price_data['annualized_daily_return'].mean()
  daily_volatility = price_data['daily_return'].std()
  annualized_volatility = daily_volatility * np.sqrt(252)
  sharpe_ratio = annualized_return / annualized_volatility

  return annualized_return,sharpe_ratio


## Import Data

In [9]:
api_key = 'your_api_key_here'

#### Symbols

In [10]:
sp_symbol_list = sp_symbols(api_key)

#### Profile

In [13]:
profile_dfs = []

for s in tqdm(sp_symbol_list):
  profile_dfs.append(import_company_data(api_key,s))

100%|██████████| 503/503 [03:07<00:00,  2.68it/s]


In [14]:
master_profile_df = pd.concat(profile_dfs)

In [15]:
master_profile_df_subset = master_profile_df[['symbol','lastDiv']]

In [201]:
div_stocks = list(master_profile_df_subset[master_profile_df_subset['lastDiv'] > 0].reset_index(drop=True)['symbol'])

## Key Metrics

In [202]:
Symbols = []
Yield_Vol = []
Years_With_Growth = []
Average_Growth = []

In [203]:
for d in tqdm(div_stocks):
  metric_data = import_key_metric_data(symbol = d,api_key = api_key)

  if int(metric_data.calendarYear.min()) > 2013:
    pass
  else:
    metric_data_subset = metric_data[["symbol","calendarYear","date","dividendYield","payoutRatio"]]
    div_history_df = import_div_history(symbol = d,api_key = api_key)
    if pd.to_datetime(div_history_df.date).min() < pd.Timestamp('2013-01-01'):
      div_history_df['date'] = pd.to_datetime(div_history_df['date'])
      div_history_df['year'] = div_history_df['date'].dt.year
      div_history_df_subset = div_history_df[['dividend','paymentDate']]

      merged_data = merge_metric_and_div_data(metric_data = metric_data_subset,div_data = div_history_df_subset)
      grouped_merged_data = merged_data.groupby('calendarYear')[['dividendYield','dividend']].sum()
      last_10 = grouped_merged_data[grouped_merged_data.index.isin(list(grouped_merged_data.index[-12:-1]))]

      Symbols.append(d)
      Yield_Vol.append(last_10.dividendYield.std())
      Years_With_Growth.append(sum(last_10.dividend.pct_change() > 0))
      Average_Growth.append(last_10.dividend.pct_change().mean())
    else:
      pass

100%|██████████| 405/405 [07:06<00:00,  1.05s/it]


In [204]:
DIV_DF = pd.DataFrame({
    'Symbol':Symbols,
    'Yield_Vol':Yield_Vol,
    'Years_With_Growth':Years_With_Growth,
    'Average_Growth':Average_Growth})

In [205]:
DIV_DF_Consistent_Growth = DIV_DF[DIV_DF['Years_With_Growth'] == 10]

In [206]:
DIV_DF_Consistent_Growth.sort_values(by='Yield_Vol',ascending=True).head()

,Symbol,Yield_Vol,Years_With_Growth,Average_Growth
90,ROP,0.000746,10,0.150504
207,UNH,0.001352,10,0.215464
141,DGX,0.001388,10,0.090408
60,JBHT,0.001510,10,0.114020
31,JKHY,0.001525,10,0.142559


In [207]:
DIV_DF_Consistent_Growth.sort_values(by='Average_Growth',ascending=False).head()

,Symbol,Yield_Vol,Years_With_Growth,Average_Growth
170,VMC,0.003117,10,0.713642
264,BAC,0.008513,10,0.443008
65,AVGO,0.010327,10,0.396381
35,HII,0.006107,10,0.287763
178,RF,0.014467,10,0.259158


## Comparison

In [212]:
div_stocks = list(DIV_DF['Symbol'])

Returns = []
Sharpe = []

for d in tqdm(div_stocks):
  r,s = annualized_returns_and_sharpe(symbol = d,api_key=api_key)
  Returns.append(r)
  Sharpe.append(s)

100%|██████████| 339/339 [03:33<00:00,  1.59it/s]


In [214]:
DIV_DF['Returns'] = Returns
DIV_DF['Sharpe'] = Sharpe

In [224]:
DIV_DF.sort_values(by='Returns',ascending=False).head()

,Symbol,Yield_Vol,Years_With_Growth,Average_Growth,Returns,Sharpe
151,NVDA,0.007714,6,-0.019400,0.836600,1.593135
189,KLAC,0.073571,9,0.923312,0.511537,1.146551
15,MPWR,0.003535,7,0.139048,0.495282,0.973710
279,LLY,0.008251,9,0.089035,0.483025,1.539669
65,AVGO,0.010327,10,0.396381,0.469736,1.244239


In [225]:
DIV_DF.sort_values(by='Sharpe',ascending=False).head()

,Symbol,Yield_Vol,Years_With_Growth,Average_Growth,Returns,Sharpe
151,NVDA,0.007714,6,-0.019400,0.836600,1.593135
279,LLY,0.008251,9,0.089035,0.483025,1.539669
65,AVGO,0.010327,10,0.396381,0.469736,1.244239
212,COST,0.024528,8,0.738914,0.283060,1.181793
173,MCK,0.002482,10,0.101936,0.335564,1.164797


In [227]:
# Step 1: Prepare the data
X = DIV_DF[['Yield_Vol', 'Years_With_Growth', 'Average_Growth']]
y = DIV_DF['Returns']

# Step 2: Add a constant to the model (intercept)
X = sm.add_constant(X)

# Step 3: Fit the multiple linear regression model
model = sm.OLS(y, X).fit()

# Step 4: Generate the summary table
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                Returns   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.704
Date:                Tue, 09 Jul 2024   Prob (F-statistic):              0.166
Time:                        18:55:49   Log-Likelihood:                 267.07
No. Observations:                 339   AIC:                            -526.1
Df Residuals:                     335   BIC:                            -510.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.1889      0.02

In [244]:
# Step 1: Prepare the data
X = DIV_DF[['Yield_Vol', 'Years_With_Growth', 'Average_Growth']]
y = DIV_DF['Returns']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the multiple linear regression model
model = sm.OLS(y, X).fit()

# Step 2: Calculate Cook's distance
influence = model.get_influence()
cooks_d = influence.cooks_distance

# The Cook's distance values are in the first element of the tuple
cooks_d_values = cooks_d[0]

# Create a DataFrame to display the Cook's distance values along with the index
cooks_d_df = pd.DataFrame({'Cook\'s Distance': cooks_d_values}, index=DIV_DF.index)

In [238]:
cooks_d_values = cooks_d[0]

# Step 3: Add Cook's distance scores back into the original DataFrame
DIV_DF['Cooks_Distance'] = cooks_d_values

DIV_DF_no_outliers = DIV_DF[DIV_DF['Cooks_Distance'] <= 0.02].reset_index(drop=True)

In [243]:
# Step 1: Prepare the data
X = DIV_DF_no_outliers[['Yield_Vol', 'Years_With_Growth', 'Average_Growth']]
y = DIV_DF_no_outliers['Returns']

# Step 2: Add a constant to the model (intercept)
X = sm.add_constant(X)

# Step 3: Fit the multiple linear regression model
model = sm.OLS(y, X).fit()

# Step 4: Generate the summary table
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                Returns   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     3.775
Date:                Tue, 09 Jul 2024   Prob (F-statistic):             0.0109
Time:                        19:52:13   Log-Likelihood:                 294.71
No. Observations:                 331   AIC:                            -581.4
Df Residuals:                     327   BIC:                            -566.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.1708      0.02